In [1]:
import chromadb
import pandas as pd
import numpy as np 
import os 
import torch
import torch.nn.functional as F
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel

In [3]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../data')

In [5]:
data = pd.read_csv(os.path.join(data_path, 'test_data', 'databricks_dollby.csv'))
print(f'data 개수: {len(data)}')
data.head(3)

data 개수: 7706


,question,best_answer,category
0,When did Virgin Australia start operating?,Virgin Australia commenced services on 31 Augu...,closed_qa
1,Why can camels survive for long without water?,Camels use the fat in their humps to keep them...,open_qa
2,"Alice's parents have three daughters: Amy, Jes...",The name of the third daughter is Alice,open_qa


In [7]:
question = data.question.values.tolist()
question[:2]

['When did Virgin Australia start operating?',
 'Why can camels survive for long without water?']

In [8]:
chroma_client = chromadb.HttpClient(host='3.39.250.201', port=8000)

In [10]:
chroma_client.list_collections()

[Collection(name=kor_emb)]

In [11]:
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [13]:
emb_collection = chroma_client.create_collection(name="eng_emb" , embedding_function=default_ef)

In [14]:
doc = []; metadata = []; ids = []

for idx in range(len(data)):
    doc.append(data.question[idx])
    metadata.append({'source': data.category[idx]})
    ids.append(str(idx + 1))

In [16]:
len(doc), len(metadata)

(7706, 7706)

In [21]:
question[:4]

['When did Virgin Australia start operating?',
 'Why can camels survive for long without water?',
 "Alice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?",
 'When was Tomoaki Komorida born?']

In [18]:
emb_collection.add(
    documents=doc,
    metadatas=metadata,
    ids=ids
)

In [23]:
txt = 'why can camels can survive in the dessert ?'

results = emb_collection.query(
    query_texts = txt, 
    n_results = 5
)
results

{'ids': [['2', '2916', '5828', '6424', '7603']],
 'distances': [[0.4543702006340027,
   0.8250446915626526,
   1.119078278541565,
   1.1853063106536865,
   1.1853063106536865]],
 'embeddings': None,
 'metadatas': [[{'source': 'open_qa'},
   {'source': 'open_qa'},
   {'source': 'closed_qa'},
   {'source': 'closed_qa'},
   {'source': 'closed_qa'}]],
 'documents': [['Why can camels survive for long without water?',
   'How can you tell the difference between a dromedary and a camel?',
   'Why did the Sahara experience a period of rapid desertification?',
   'What do giant Pandas eats?',
   'What do giant Pandas eats?']],
 'uris': None,
 'data': None}